# Route Checker

## 1. Batfishコンテナの起動

Batfishコンテナが起動しているか確認し、起動していない場合は起動する。

In [2]:
%%bash
if (docker ps -f "ancestor=batfish/batfish" | grep "0.0.0.0:9996-9997" > /dev/null); then
  echo "Batfish container is already running."
else
  docker run -d -p 9997:9997 -p 9996:9996 batfish/batfish
fi

Batfish container is already running.


## 2. startup.pyの実行

startup.pyを実行し、ライブラリのインポートなどを行う。

In [3]:
%run startup.py

/home/shino/.pyenv/versions/3.6.5/lib/python3.6/site-packages/pybatfish/client/commands.py:50: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


## 3. 通信の設定

以下のセル内の各変数に検証したい通信の情報を格納し、実行する。
HeaderConstraintsのその他の引数は以下を参照。
[https://github.com/batfish/pybatfish/blob/master/pybatfish/datamodel/flow.py#L647-L669](htps://github.com/batfish/pybatfish/blob/master/pybatfish/datamodel/flow.py#L647-L669)

In [4]:
# 送信元IPアドレス
src_ip = "192.168.10.1"
# 宛先IPアドレス
dst_ip = "192.168.20.1"
# アプリケーション
app = "http"
flow = HeaderConstraints(srcIps=src_ip,
                         dstIps=dst_ip,
                         applications=[app])

## 4 ホストの設定

以下のセル内のJSONに送信元および宛先のホスト情報を指定し、実行する。

In [5]:
import json

host_file_dir = "networks/example2/hosts/"
src_host_dict = {
                 "hostname": "src_host",
                 "hostInterfaces": {
                   "eth0": {
                     "name": "eth0",
                     "prefix": str(src_ip + "/24"),
                     }
                   }
                 }

dst_host_dict = {
                 "hostname": "dst_host",
                 "hostInterfaces": {
                   "eth0": {
                     "name": "eth0",
                     "prefix": str(dst_ip + "/24"),
                     }
                   }
                 }

f = open("{dir}src_host.json".format(dir=host_file_dir), "w")
f.write(json.dumps(src_host_dict))
f.close()

f = open("{dir}dst_host.json".format(dir=host_file_dir), "w")
f.write(json.dumps(dst_host_dict))
f.close()

## 3. 設定ファイルのアップロード

設定ファイルをBatfishコンテナへアップロードする(スナップショット名の後にいくつかのログが表示される)。

In [6]:
# ネットワークの名前を指定する
NETWORK_NAME = "datacenter1"
# ネットワークの状態の名前を指定する(current, candidateなど)
SNAPSHOT_NAME = "current"
# 設定ファイルの格納先を指定する
SNAPSHOT_PATH = "networks/example2"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'current'

## 5. 通信の検証

以下のセルを実行し、結果を確認する。

In [7]:
traceroute_ans = bfq.traceroute(
                                headers=flow,
                                startLocation="src_host"
                                #startLocation="enter(core[GigabitEthernet1/0])"
                               ).answer()
display_html(traceroute_ans.frame())

## Tips

In [9]:
traceroute_ans = bfq.traceroute(
                                headers=flow,
                                startLocation="src_host"
                                #startLocation="enter(core[GigabitEthernet1/0])"
                               ).json()
print(traceroute_ans)

{
  "class": "org.batfish.question.traceroute.TracerouteQuestion",
  "differential": false,
  "headers": "${headers}",
  "ignoreAcls": "${ignoreAcls}",
  "instance": {
    "description": "Trace the path(s) for the specified flow.",
    "instanceName": "traceroute_2c858df0-db2b-4597-8d45-ba4424564e27",
    "longDescription": "This question performs a virtual traceroute in the network from a starting node. A destination IP and ingress (source) node must be specified. Other IP headers are given default values if unspecified.\nUnlike a real traceroute, this traceroute is directional. That is, for it to succeed, the reverse connectivity is not needed. This feature can help debug connectivity issues by decoupling the two directions.",
    "tags": [
      "dataPlane",
      "reachability",
      "traceroute"
    ],
    "variables": {
      "headers": {
        "description": "Packet header constraints.",
        "optional": true,
        "type": "headerConstraint",
        "value": {
        